# 양정고등학교 HRD 메이커 퀀트 실습 2
## 코스피/코스닥 전종목 주가데이터 수집
[Finance Data Reader 공식 문서](https://financedata.github.io/posts/finance-data-reader-users-guide.html)


### 1. 초기 세팅




In [1]:
! pip install finance-datareader -q # 필요한 모듈 설치

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 600.3 kB/s eta 0:00:00


In [2]:
! mkdir data && mkdir data/stockdata

In [3]:
import FinanceDataReader as fdr # 필요한 모듈 불러오기
import pandas as pd
import concurrent.futures
import time
import datetime
import shutil

from IPython.display import display, update_display

### 2. 코스피 및 코스닥 종목 정보 수집

In [4]:
# 종목 목록 가져오기
stock_list = fdr.StockListing('KRX')

In [5]:
# 코넥스 종목은 제외하기
stock_list = stock_list.loc[stock_list['Market'] .isin(["KOSPI","KOSDAQ"])]

In [6]:
# 지역이 있는 종목만 필터링하기
stock_name_list = stock_list['Name'].tolist() # 선물, 우선주 등이 지역 정보가 없음

In [7]:
stock_name_list=stock_list.loc[~stock_list['Name'].str.endswith('우'),'Name'].tolist()

In [8]:
# 종목 정보 내보내기]
stock_list.to_csv("./data/2025stock.csv", sep = "\t", index = False, encoding = "utf-8")

In [9]:
# 내보낸 정보 출력하기
file_path = "./data/2025stock.csv"
stock_list = pd.read_csv(file_path, sep="\t", encoding="utf-8")
print(stock_list)

        Code        ISU_CD      Name  Market   Dept    Close  ChangeCode  \
0     005930  KR7005930003      삼성전자   KOSPI    NaN    55850           1   
1     000660  KR7000660001    SK하이닉스   KOSPI    NaN   182700           1   
2     373220  KR7373220003  LG에너지솔루션   KOSPI    NaN   341500           3   
3     207940  KR7207940008  삼성바이오로직스   KOSPI    NaN  1034000           2   
4     005380  KR7005380001       현대차   KOSPI    NaN   189200           1   
...      ...           ...       ...     ...    ...      ...         ...   
2704  002787  KR7002782001   진흥기업2우B   KOSPI    NaN     9550           1   
2705  004415  KR7004411005     서울식품우   KOSPI    NaN     1220           2   
2706  032685  KR7032681009     소프트센우  KOSDAQ  중견기업부     6970           1   
2707  001527  KR7001522002     동양2우B   KOSPI    NaN     8020           1   
2708  001525  KR7001521004       동양우   KOSPI    NaN     3945           1   

      Changes  ChagesRatio     Open     High      Low   Volume        Amount  \
0      

#### 3. 코스피 및 코스닥 전종목 주가 데이터 수집

In [10]:
# Widget 생성
progress_widget = display("작업 시작중...", display_id=True)

def collect_stock_data(code, name):
    try:
        data = fdr.DataReader(code, "2020-01-01", "2025-04-01")
        if len(data) > 300:
            data.to_csv("./data/stockdata/{}.csv".format(name))
        # 진행 상황 업데이트
        progress_widget.update(f"정보 수집중 - {name}...")
    except Exception as e:
        # 에러 발생 시, 진행 상황 업데이트
        progress_widget.update(f"Error occurred while collecting data for {name}: {str(e)}")

# 동시에 처리할 작업 수
max_workers = 10

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # 각 종목에 대해 collect_stock_data 함수를 병렬 실행
    futures = [executor.submit(collect_stock_data, code, name) for code, name in stock_list[['Code', "Name"]].values]

    # 실행 결과 확인
    for future in concurrent.futures.as_completed(futures):
        future.result()  # 결과를 가져올 필요는 없지만, 에러 발생 시 예외 처리를 위해 호출

# 모든 작업 완료 후 메시지 출력
progress_widget.update("모든 작업 완료!")

'모든 작업 완료!'

In [11]:
# 압축할 디렉토리 경로
directory_to_zip = "./data/stockdata"

# 압축된 파일 이름
zip_file_name = "zipped"

# 디렉토리 내부 모든 파일 압축하기
shutil.make_archive(zip_file_name, 'zip', directory_to_zip)
print(f"압축 파일 {zip_file_name} 이 생성 완료!")

압축 파일 zipped 이 생성 완료!
